<a href="https://colab.research.google.com/github/bimarshak7/chat-bots/blob/main/General_Convo_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#snippet to mount drive and copy kaggle API keys
# palge you kaggle keys at MyDrive/Colab Notebooks/ as kaggle.json
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle.json" ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [ ]:
!kaggle datasets download -d kreeshrajani/3k-conversations-dataset-for-chatbot

  0% 0.00/67.1k [00:00<?, ?B/s]
100% 67.1k/67.1k [00:00<00:00, 69.7MB/s]


In [ ]:
!unzip -q *.zip -d data/

In [ ]:
import json
import pandas as pd
from collections import Counter
import torch
import random
from torch import nn
import torch.nn.functional as F
from torchtext.vocab import vocab
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import math
import string

In [ ]:
DATA_PATH = "/content/data/Conversation.csv"

In [ ]:
df = pd.read_csv(DATA_PATH)
df.sample(5)

,Unnamed: 0,question,answer
2256,2256,you must be joking.,"no, i'm serious."
2031,2031,"well, you can't play baseball in a rainstorm.",i thought they were playing under a dome.
1095,1095,that's a good idea.,i found a button in my pant cuffs one time.
785,785,we haven't been in a while.,we haven't been in a month.
2954,2954,"yes, i guess most of us are stuck in a routine.",i wonder if there is any job that you don't re...


In [ ]:
df["question"] = df["question"].apply(lambda x: x.lower())
df["answer"] = df["answer"].apply(lambda x: x.lower())

In [ ]:
df["question"] = df["question"].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df["answer"] = df["answer"].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
counter = Counter()
for i in range(len(df)):
    row = df.iloc[i]
    counter.update(row["question"].split())
    counter.update(row["answer"].split())

In [ ]:
vocab_en = vocab(counter, min_freq=5, specials=('<UNK>', '<BOS>', '<EOS>', '<PAD>'))

In [ ]:
vocab_en.set_default_index(vocab_en['<UNK>'])

In [ ]:
df["question"] = df["question"].apply(lambda x: "<BOS> "+x+" <EOS>")
df["answer"] = df["answer"].apply(lambda x: "<BOS> "+x+" <EOS>")

In [ ]:
MAX_LEN_QN = df["question"].apply(lambda x: len(x.split())).max()
MAX_LEN_ANS = df["answer"].apply(lambda x: len(x.split())).max()
MAX_LEN_QN,MAX_LEN_ANS

(21, 21)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.X = df["question"]
        self.y = df["answer"]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        qn = self.X.iloc[idx]
        ans = self.y.iloc[idx]

        qn_indices = [vocab_en[word] for word in qn.split()]

        ans_indices = [vocab_en[word] for word in ans.split()]

        return torch.tensor(qn_indices), len(qn_indices), torch.tensor(ans_indices)


In [ ]:
ds = MyDataset(df)

In [ ]:
train_ds,test_ds = torch.utils.data.random_split(ds, [0.8, 0.2])

In [ ]:
BATCH_SIZE = 32

In [ ]:
def my_collate(batch):
    # Extract sequences and targets
    qns = [item[0] for item in batch]
    src_lens = [item[1] for item in batch]
    ans = [item[2] for item in batch]
    # Pad sequences
    padded_qn = pad_sequence(qns, batch_first=True, padding_value=vocab_en["<PAD>"])
    padded_ans = pad_sequence(ans, batch_first=True, padding_value=vocab_en["<PAD>"])

    # Return padded sequences and targets
    return padded_qn, torch.tensor(src_lens), padded_ans

In [ ]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,collate_fn=my_collate)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE,collate_fn=my_collate)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim,2, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_len):

        #src = [src len, batch size]
        #src_len = [batch size]

        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]

        #need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'),enforce_sorted=False)

        packed_outputs, hidden = self.rnn(packed_embedded)

        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch

        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)

        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros

        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]

        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer

        #hidden [-2, :, : ] is the last of the forwards RNN
        #hidden [-1, :, : ] is the last of the backwards RNN

        #initial decoder hidden is final hidden state of the forwards and backwards
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]

        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_outputs, mask):

        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]

        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))

        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)

        #attention = [batch size, src len]

        attention = attention.masked_fill(mask == 0, -1e10)

        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs, mask):

        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]

        input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        a = self.attention(hidden, encoder_outputs, mask)

        #a = [batch size, src len]

        a = a.unsqueeze(1)

        #a = [batch size, 1, src len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        weighted = torch.bmm(a, encoder_outputs)

        #weighted = [batch size, 1, enc hid dim * 2]

        weighted = weighted.permute(1, 0, 2)

        #weighted = [1, batch size, enc hid dim * 2]

        rnn_input = torch.cat((embedded, weighted), dim = 2)

        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]

        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

        #prediction = [batch size, output dim]

        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device

    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask

    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):

        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        src = src.permute(1,0)
        trg = trg.permute(1,0)

        # print("SRC SHAPE",src.shape)
        # print("TRG SHAPE",trg.shape)

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        mask = self.create_mask(src)

        #mask = [batch size, src len]

        for t in range(1, trg_len):

            #insert input token embedding, previous hidden state, all encoder hidden states
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_en)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.4
DEC_DROPOUT = 0.3
SRC_PAD_IDX = vocab_en["<PAD>"]
ENCODER_N_LAYERS = 2

In [ ]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = vocab_en["<PAD>"])

In [ ]:
# def init_weights(m):
#     for name, param in m.named_parameters():
#         if 'weight' in name:
#             nn.init.normal_(param.data, mean=0, std=0.01)
#         else:
#             nn.init.constant_(param.data, 0)

# model.apply(init_weights)

In [ ]:
# model.load_state_dict(torch.load("/kaggle/input/seq2seqmental/seq2seq(mental).pt"))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=2e-4)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = vocab_en["<PAD>"])

In [ ]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0
    pbar = tqdm(iterator)
    for i, batch in enumerate(pbar):

        src = batch[0].to(device)
        src_lens = batch[1]
        trg = batch[2].to(device)
        optimizer.zero_grad()

        output = model(src, src_lens, trg)

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        trg = trg[:,1:].reshape(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
#         print("OP shape ",output.shape)
#         print("TRG Shape ",trg.shape)
        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
#         if i%500==0:print("Batch:",epoch_loss)

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        pbar = tqdm(iterator)
        for i, batch in enumerate(pbar):

            src = batch[0].to(device)
            src_lens = batch[1]
            trg = batch[2].to(device)
            output = model(src, src_lens, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[:,1:].reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
model = model.to(device)

In [ ]:
N_EPOCHS = 1000
CLIP = 70.0

In [ ]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_loader, criterion)

    torch.save(model.state_dict(), "seq2seq.pt")

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    print(f'\tEPOCH:{epoch+1}/{N_EPOCHS}\tTrain Loss: {train_loss:.3f} |\t Val. Loss: {valid_loss:.3f}')

100%|██████████| 24/24 [00:00<00:00, 58.65it/s]


	EPOCH:1/1000	Train Loss: 5.474 |	 Val. Loss: 5.284


100%|██████████| 24/24 [00:00<00:00, 58.70it/s]


	EPOCH:2/1000	Train Loss: 5.204 |	 Val. Loss: 5.270


100%|██████████| 24/24 [00:00<00:00, 42.65it/s]


...

100%|██████████| 24/24 [00:00<00:00, 53.83it/s]


	EPOCH:996/1000	Train Loss: 0.043 |	 Val. Loss: 13.249


100%|██████████| 24/24 [00:00<00:00, 43.66it/s]


	EPOCH:997/1000	Train Loss: 0.044 |	 Val. Loss: 13.230


 17%|█▋        | 16/94 [00:00<00:04, 16.67it/s]

In [ ]:
# torch.save(model.state_dict(), "seq2seq.pt")

In [ ]:
# model.load_state_dict(torch.load("seq2seq.pt",map_location=torch.device('cpu')))

In [ ]:
model = model.eval()

In [ ]:
def tokenize(qn):
    qn = qn.lower()
    qn = qn.translate(str.maketrans('', '', string.punctuation))
    indices = [vocab_en[word] for word in qn.split()]
    indices.insert(0,vocab_en["<BOS>"])
    indices.append(vocab_en["<EOS>"])
    ip_tensor = torch.tensor(indices).unsqueeze(0).permute(1,0)

    return ip_tensor

In [ ]:
def make_pred(qn,trg_len=12):
    src = tokenize(qn).to(device)
    encoder_outputs, hidden = model.encoder(src, torch.tensor([len(src)]))
    trg = torch.tensor([[vocab_en["<BOS>"]]])

    with torch.no_grad():
        trg_ip = trg[0,:].to(device)
        trg_vocab_size = model.decoder.output_dim
        outputs = torch.zeros(trg_len, 1, trg_vocab_size).to(device)
        mask = model.create_mask(src).to(device)

        for t in range(1, trg_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = model.decoder(trg_ip, hidden, encoder_outputs, mask)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #get the highest predicted token from our predictions
            trg_ip = output.argmax(1)

        outputs = outputs.squeeze(1)
        return " ".join([vocab_en.get_itos()[i] for i in outputs.argmax(1) if i not in [0,1,2]])

In [ ]:
while True:
    you = input("YOU > ")
    if you=="q" or you=="quit":
        break

    bot = make_pred(you)
    print(f"BOT > {bot}")
print(f"BOT > Bye! Have a good day dear.")

YOU > hello
BOT > to it it blood blood blood thats a
YOU > no blood
BOT > change change oh but but then all eight eight
YOU > no change 
BOT > blue ticket later but be i rush it eight
YOU > do you have a ticket?
BOT > you you no i you see see see see the the
YOU > no you
BOT > i i you i rush now you
YOU > yes you
BOT > to houses did we we with hell that
YOU > we we
BOT > as why nothing shoes shoes of believe on on dont
YOU > why nothing?
BOT > landed going too call know know around over
YOU > do not call
BOT > the the for just just didnt didnt one have what
YOU > have one
BOT > all be im i think
YOU > bye
BOT > to it it blood blood blood thats a
YOU > hi
BOT > to it it blood blood blood thats a
YOU > quit
BOT > Bye! Have a good day dear.


Little baby trying to learn words!!! 😆😆